### Name :  W16 作業2
### Function : Application of NER and Topic Model 

#### input data : 蔣萬安.csv

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import gc
import math
import jieba
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

##  function Extract_NER():   Extract NER from receiving text list

In [3]:
def Extract_NER(ls_textRow):
    from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
    
    # 載入 自訂詞典
    User_Dict = {}
    with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
        for udic in UDicts:
            udWord = udic.strip().split(" ")
            if len(udWord) > 1:
                User_Dict[udWord[0]] = udWord[1]
            else:
                User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
    dictionary = construct_dictionary(User_Dict)
 
    ws = WS("./data")
    pos = POS("./data")
    ner = NER("./data")
    
    #  ls_segRow = ws(ls_textRow) ------------   without User Dictionary
    ls_segRow = ws(ls_textRow, 
                   sentence_segmentation=True,
                   segment_delimiter_set = {",", "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    #print("ls_segRow = ", ls_segRow)
    #print("-----------------------------")
    
    del ws
    del WS
    gc.collect()
    ls_posRow = pos(ls_segRow)
    del pos
    del POS
    gc.collect()
    ls_nerRow = ner(ls_segRow,ls_posRow)
    del ner
    del NER
    gc.collect()
    #print("ls_nerRow = ", ls_nerRow)
    #print("-----------------------------")
    return ls_nerRow

## 1. Read data from csv or xlsx file

In [4]:
df = pd.read_csv('蔣萬安.csv',encoding='utf-8') #匯資料
df.head(1)

,Unnamed: 0,屆期,會期,會議日期,委員姓名,逐字稿
0,9388,9,1,3/4/2016,蔣萬安,﻿蔣委員萬安：（15 時 59 分）主席、行政院張院長、杜副院長、各部會首長、各位同仁。今天...


## 2.  Main function:
### 2.1  Pass text data to function "Extract_NER" by row
### 2.2  Integrate every 10 NER return rows into one data frame  
### 2.3  Write frame into small csv after droping duplicate, locating class, filtering short NER, sorting by class.. 

In [6]:
#逐筆傳遞欄位 "text"
chiang_text = df["逐字稿"]
print("逐字稿 欄位筆數 = ",len(chiang_text))
print("------------------------------------------")
ls_class = list()
ls_ner = list()
fileCunt = 0
rowCunt = 0
for i in range(len(chiang_text)):
    rowCunt += 1
    ls_dfRow = [chiang_text[i]]
    # print("ls_dfRow = ", ls_dfRow)
    print("Row No = ",str(i))
    entity_list = Extract_NER(ls_dfRow)
    for j in range(len(entity_list[0])):
        seg = entity_list[0].pop()
        # print("seg = ",seg)
        ls_class.append(seg[2])
        ls_ner.append(seg[3])
    #print("===========================================")
    #print("ls_class = ",ls_class)
    #print("ls_ner = ",ls_ner)
    # if i == 5 or rowCunt == 2:
    if i == (len(chiang_text)-1) or rowCunt == 10:
        fileCunt += 1
        extract_df = pd.DataFrame({'Class':ls_class, 'NER':ls_ner})
        deDup_df = extract_df.drop_duplicates().copy()
        del extract_df #去除重複
        cate_df = deDup_df.loc[deDup_df['Class'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','NORP'])].copy()
        del deDup_df
        filt_df = cate_df[cate_df['NER'].map(len) >= 2].copy()
        del cate_df
        NER_df = filt_df.sort_values(by=['Class']).copy()
        del filt_df
        #print("__________________________________________________________")  
        #print(NER_df)
        print("fileCunt No = ",str(fileCunt))
        NER_df.to_csv(r"Chiang Ner out/Chiang_" + str(fileCunt)+ ".csv")
        #NER_df.to_csv(r"ner out/NerTest_" + str(fileCunt)+ ".csv")
        rowCunt = 0
print("=====================================================================")
print("Totle NER Files = ",fileCunt)


逐字稿 欄位筆數 =  366
------------------------------------------
Row No =  0


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  1
Row No =  2
Row No =  3
Row No =  4
Row No =  5
Row No =  6
Row No =  7
Row No =  8
Row No =  9
fileCunt No =  1
Row No =  10


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  11
Row No =  12
Row No =  13
Row No =  14
Row No =  15
Row No =  16
Row No =  17
Row No =  18
Row No =  19
fileCunt No =  2
Row No =  20


...

Row No =  341
Row No =  342
Row No =  343
Row No =  344
Row No =  345
Row No =  346
Row No =  347
Row No =  348
Row No =  349
fileCunt No =  35
Row No =  350


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  351
Row No =  352
Row No =  353
Row No =  354
Row No =  355
Row No =  356
Row No =  357
Row No =  358
Row No =  359
fileCunt No =  36
Row No =  360


C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Row No =  361
Row No =  362
Row No =  363
Row No =  364
Row No =  365
fileCunt No =  37
Totle NER Files =  37


## 3. Integrate all csv files into single data frame

In [3]:
import glob
# import pandas as pd

path = r'Chiang Ner out'
#path = r'ner out'
allFiles = glob.glob("{}/*.csv".format(path))
li = []
for filename in allFiles:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

nerFrame = pd.concat(li, axis=0, ignore_index=True)
print(len(nerFrame))
nerFrame.groupby('Class').count()

28109


,NER
Class,
EVENT,650
GPE,5252
LAW,1790
LOC,1174
NORP,996
ORG,10436
PERSON,7811


## 4. Drop duplicates, group by class and write into csv file 

In [4]:
ddFrame = nerFrame.drop_duplicates().copy()
print(len(ddFrame))
ddFrame.groupby('Class').count()

1920


,NER
Class,
EVENT,60
GPE,321
LAW,120
LOC,118
NORP,64
ORG,737
PERSON,500


In [9]:
ddFrame.sort_values(by=['Class']).to_csv(r"Chiang Ner out/Ligis_NER.csv")
#ddFrame.sort_values(by=['Class']).to_csv(r"ner out/NerTest.csv")

In [10]:
df = pd.read_csv('蔣萬安.csv',encoding='utf-8') #匯資料
df.head(1)

,Unnamed: 0,屆期,會期,會議日期,委員姓名,逐字稿
0,9388,9,1,3/4/2016,蔣萬安,﻿蔣委員萬安：（15 時 59 分）主席、行政院張院長、杜副院長、各部會首長、各位同仁。今天...


#### 用蔣萬安的資料跑TOPIC MODEL

In [11]:
text_list = list(df['逐字稿']) # 把所有message轉成一個list 用for iterate

chiang_str = ','.join(text_list)
print(chiang_str) #轉字串斷詞

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import gc
import tensorflow as tf
# import keras

# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 100    # 未給定權重者一律賦予預設值 100               
dictionary = construct_dictionary(User_Dict)

In [13]:
# 載入模型
ws = WS("./data")
# pos = POS("./data")
# ner = NER("./data")

C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [14]:
# 載入 StopWord
stopword_list = []
with open("KCC Data/Dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())

In [15]:
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced
strCut = ws([chiang_str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                 coerce_dictionary = dictionary)

print("------------CKIPTagger Word Segment ws(chiang_str,....) + 刪除標點及停用字 ---------------")
stayed_lst= [] #斷好字的
for word in strCut[0]:
     # 刪除停用字
    if word not in stopword_list:
        stayed_lst.append(word)
        
print(" ".join(stayed_lst))

------------CKIPTagger Word Segment ws(chiang_str,....) + 刪除標點及停用字 ---------------
﻿ 蔣 委員 萬安 15   時  59   分 行政院 張 院長 杜 副院長 部會 首長 位 同仁 席
 民眾 關心 議題 提出 質詢 院長  Google  服務 本席 
 矽谷 擔任 執業 律師 服務 事務所 協助  Google  新創 公司 正
 式  IPO  上市 院長 加入 內閣 政務 委員 科技部 長 副院長 院長 
 席 院長 深 期待 院長 秉持 科技 專業 產業 台灣 
 發揮 優勢 帶領 台灣 國際 間 提升 競爭力 台灣 院長 突破 
 困境 翻轉 台灣 經濟 科技 科技 人才 資金 年 行政院 一
 直 推動 新創 產業 過時 需要 法規 鬆綁 三方 支付 
 專法 創業家 簽證 去年 立法院 修訂 公司 法 增訂 閉鎖型 專章 特別 新創 公司 
 設立 請教 院長 去年 推動 增訂 閉鎖型 專章 目的 
 請 行政院 張 院長 答復 
 張 院長 善政 16   時  1  分 委員 上市 上櫃 公司 更多 資金 股東 
 彈性 
 蔣 委員 萬安 當初 公司 法 增訂 閉鎖型 專章 目的 新創 團隊 台灣
 設立 公司 多元 彈性 籌資 工具 發行 轉換 公司 債 特別股 
 發行 公司 投資 之間 設定 權利 義務 關係 年輕 創業家 現金 外
 技術 勞務 信用 換取 股權 請教 院長 部長 去年  6 月 三讀通過 公司 法
 修訂 閉鎖型 專章  2  月 底 總共 家 閉鎖型 公司 成立 
 鄧 部長 振中 兩 月 前 同仁 數字 二十幾 家 更多 公司 成立 
 
 張 院長 善政 好像  60  家 
 蔣 委員 萬安 統計 去年  6 月  2 月 底 總共  62  家 請教 院長 部長 數
 字 少 
 張 院長 善政 會期 更多 
 蔣 委員 萬安 統計 去年  6 月  2 月 底 台灣 設立 公司  2 萬 7 000   家 閉鎖型 公司 
  62  家 占 比例 不到 千分之二 百分之零點二 去年 推動 閉鎖型 公司 專章 
 預期 外界 期待 落差 請問 院長 原因 
 張 院長 善政 觀念 大部分 成立 公司 團隊 還不了 解 型態

In [16]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [17]:
fList = []
with open("蔣萬安_output.txt","r", encoding='utf-8-sig') as files:
     for file in files:
        #simple_preprocess: tokenize,去除標點 - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15) 
        fList.append(list(utils.simple_preprocess(file, deacc=True, min_len=2,)))
print("fList[:2] = ", fList[:2])
print("------------------------------")

fList[:2] =  [['jieba_results'], ['委員', '萬安', '主席', '行政院', '院長', '杜副', '院長', '部會首長', '各位', '同仁', '今天', '本席', '非常高興', '可以', '針對', '民眾', '關心', '議題', '提出', '質詢', '院長', '過去', 'google', '服務', '本席', '過去', '矽谷', '擔任', '執業', '律師', '過去', '服務', '過的', '事務所', '剛好', '協助', 'google', '一個', '新創', '公司', 'ipo', '上市', '所以', '當我', '知道', '院長', '加入', '內閣', '政務委員', '科技部長', '院長', '一直', '院長', '院長', '期待', '相信', '院長', '秉持', '過去', '科技', '專業', '以及', '產業', '了解', '台灣', '發揮優勢', '進而', '帶領', '台灣', '國際', '提升', '競爭力', '台灣', '未來', '相信', '院長', '知道', '我們', '突破', '現在', '困境', '翻轉', '台灣', '經濟', '就是', '科技', '科技', '不外乎', '人才', '資金', '過去', '幾年', '行政院', '直在', '推動', '許多', '新創', '產業', '尤其', '許多', '過時', '需要', '一些', '限制', '進行', '法規', '第三方', '支付', '專法', '創業家', '簽證', '尤其', '去年', '立法院', '修訂', '公司法', '增訂', '閉鎖', '專章', '特別', '針對', '新創', '公司', '設立', '請教', '院長', '去年', '我們', '積極', '推動', '增訂', '閉鎖', '專章', '主要', '目的', '為何', '主席', '行政院', '院長', '院長', '善政', '主席', '各位', '委員', '一些', '沒有', '上市', '上櫃', '公司', '資金', '股東', '彈性', '委員', '萬安', '其實', '當初', '

In [18]:
dict = corpora.Dictionary(fList)
print("dict = ", dict)
print("------------------------------")

dict =  Dictionary(20714 unique tokens: ['jieba_results', 'google', 'ipo', '一下', '一五']...)
------------------------------


In [19]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in fList]   
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:1]]
print("======================================================")

corpus[:1] =  [[(0, 1)]]
------------------------------
print lists of [word:frequency]
('jieba_results', 1)


In [20]:
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
# print(ldamodel.print_topics())   列出最重要的前若干个主题

output = open("KCC Data/NewsCisTest-KccDict2020-LDA-TopicNum10_chiang.txt", 'w', encoding='utf-8-sig')
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1)
        output.write(str1)
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(fList), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda)

strTM =  00.071*"公報" + 0.041*"記錄" + 0.000*"查尋" + 0.000*"bnt" + 0.000*"藝文" + 0.000*"東洋" + 0.000*"限塑" + 0.000*"參採" + 0.000*"坐月子" + 0.000*"帶走" + 0.000*"澤民" + 0.000*"擅帶" + 0.000*"丁怡" + 0.000*"登機" + 0.000*"預算書" + 0.000*"離異" + 0.000*"胎盤" + 0.000*"美牛" + 0.000*"covax" + 0.000*"來訪"

strTM =  10.000*"區或" + 0.000*"劃設" + 0.000*"其談" + 0.000*"公分" + 0.000*"做調" + 0.000*"九項" + 0.000*"九孔" + 0.000*"之詞" + 0.000*"並陳" + 0.000*"陳水扁" + 0.000*"讀準" + 0.000*"脫掉" + 0.000*"而調" + 0.000*"給幼" + 0.000*"私幼" + 0.000*"的準" + 0.000*"畫上" + 0.000*"區是" + 0.000*"區嗎" + 0.000*"漲幅"

strTM =  20.005*"有當" + 0.005*"公決" + 0.005*"本院" + 0.004*"敬請" + 0.004*"度假" + 0.003*"擬函轉" + 0.002*"濟世" + 0.002*"黨為" + 0.002*"大道" + 0.002*"慈悲" + 0.002*"情案" + 0.002*"申辦" + 0.001*"號等" + 0.001*"輸台" + 0.001*"集會遊行" + 0.001*"雙重" + 0.001*"無異議" + 0.001*"案擬" + 0.001*"堤防" + 0.001*"贊成者"

strTM =  30.004*"天使" + 0.003*"閉鎖" + 0.002*"投資人" + 0.002*"善政" + 0.002*"創業家" + 0.001*"投資者" + 0.001*"所得稅" + 0.000*"草創" + 0.000*"業師" + 0.000*"台矽" + 0.000*"億美金" + 0.000*"重整" + 0.000*"熱線"

In [21]:
import pyLDAvis
import pyLDAvis.gensim #import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [22]:
pyLDAvis.enable_notebook
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict, doc_topic_dist=None)
"""
参数说明：
prepare(topic_model, corpus, dictionary, doc_topic_dist=None)
topic_model：训练得到的 gensim LdaModel 对象，不支持其他 gensim 模型类型
corpus：以词袋形式表示的语料（用于训练主题模型的语料）
dictionary：用于构建词袋模型的字典，即 gensim Dictionary 对象
doc_topic_dist：可选参数，用于传入 LDA 模型的文档主题分布，默认取值为 None，当需要多次调用 prepare 函数时，可以传入该参数
"""
pyLDAvis.display(vis)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
